# BLOCK-T623 Dynamic Tests Analysis - 20% at 40 elevation

In [ ]:
BLOCK_ID = "BLOCK-T623"
START_TIME = "2025-11-01 00:46:00"
END_TIME = "2025-11-01 00:56:00"

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re

from astropy.time import Time
from astropy import units as u
from datetime import datetime

from lsst.summit.utils.blockUtils import BlockParser
from lsst.summit.utils.efdUtils import getDayObsForTime, makeEfdClient
from lsst.summit.utils.tmaUtils import TMAEvent, TMAEventMaker, TMAState

import BLOCK_T227_utils as block

# Create a client to access the Engineering Facility Database
efd_client = makeEfdClient()

# Create an object that mines TMA Slew Events
event_maker = TMAEventMaker()

In [ ]:
print(
    f"\nQuery data for {BLOCK_ID}"
    f"\n  starts at {START_TIME} and"
    f"\n  ends at {END_TIME}\n"
)

## Querying data

In [ ]:
start_time = Time(START_TIME)
end_time = Time(END_TIME)

Since `block_parser` is not working as expected, I will have to find slew events withing the range above manually.

In [ ]:
day_obs = getDayObsForTime(start_time)
print(f"Query TMA events in day_obs = {day_obs}")

events = event_maker.getEvents(day_obs)
print(f"Collected {len(events)} TMA events")

In [ ]:
# We are working with lists 
mask_begin = np.array([evt.begin >= start_time for evt in events])
mask_end = np.array([evt.end <= end_time for evt in events])
mask = mask_begin & mask_end

events = np.array(events)
filtered_events = events[mask]

print(f"Found {filtered_events.size} TMA Slew Events")

The cell above shows how, somehow, we are not grabbing all the relevant information from the Script and Scheduler events. Something is missing. Let's keep going. 

In [ ]:
df_hp_forces = block.get_hp_minmax_forces(efd_client, filtered_events, verbose=False)

In [ ]:
df_hp_forces

Alright, those values make sense. Now let's plot the histogram. There is not much data. But we should see something. 

In [ ]:
%matplotlib inline 
block.plot_histogram_hp_minmax_forces(df_hp_forces, day_obs, BLOCK_ID)